In [ ]:
!git clone https://github.com/adarsh1001/Hand_Segmentation_RefineNet
!cp /content/Hand_Segmentation_RefineNet/RefineNet/refinenet.py -r /content
!cp /content/Hand_Segmentation_RefineNet/RefineNet/weights -r /content
!cp /content/drive/MyDrive/handmask/hands.pth.tar -r /content/weights


Cloning into 'Hand_Segmentation_RefineNet'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.


In [ ]:
!pip install py7zr
import os
import glob
from pathlib import Path
from csv import writer
import cv2
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line, VideoWriter_fourcc, VideoWriter
from numpy import asscalar, ndarray, array, zeros, concatenate, load, uint8, set_printoptions, inf
from pathlib import Path
from shutil import rmtree
import py7zr
import random
from os.path import split
import matplotlib.pyplot as plt
import re
import torch
from refinenet import refinenet
from tqdm import tqdm
def atoi(strr):
    return int(strr) if strr.isdigit() else strr

def natural_keys(strr):
    return [ atoi(c) for c in re.split('(\d+)', strr) ]
class Writer:
    temp_path = ''
    def write_to_csv(self, result: ndarray, out_csv: str):
        out_csv = Path(out_csv)
        with out_csv.open('a+') as csv:
            csv_writer = writer(csv)
            csv_writer.writerow(result)
    def write_as_video(self, path, image):
        if self.temp_path != path: 
            self.temp_path = path
            self.writer = VideoWriter(path, VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))
        self.writer.write(image)  
w = Writer()
img_scale = 1./255
img_mean = array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
img_std = array([0.229, 0.224, 0.225]).reshape((1, 1, 3))
numC = 7
cmap = load('weights/cmap.npy')
net = refinenet(numC, pretrained=True).eval()
net = net.cuda() #Run only with Cuda support!
pbar = tqdm(total=300, bar_format ='{l_bar}{bar}|  {n_fmt}/{total_fmt}  [ Elapsed Time : {elapsed} | ETA : {remaining} ]', leave=True, position=0)
make_dir = Path('/content/Output')
if not make_dir.exists():make_dir.mkdir()

make_dir = Path('/content/temp')
if not make_dir.exists():make_dir.mkdir()  

In [ ]:
from google.colab.patches import cv2_imshow
current_ds = '/content/Age_2.avi'
label = 'Age'
# make_dir = Path('/content/Output/'+label)
# if not make_dir.exists():make_dir.mkdir()
print(label)
video = current_ds
video_cap = VideoCapture(str(video))
flag, frame = video_cap.read()
cnt =1
frame_length = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
rmtree('/content/temp')
Path('/content/temp').mkdir()
while flag:
    cv2.imwrite('/content/temp/frame%d.jpg' %cnt, frame)
    flag, frame = video_cap.read()
    cnt+=1
imgs = os.listdir('/content/temp')
imgs.sort(key=natural_keys)
for imgname in imgs:
    print(imgname)
    img = array(plt.imread('/content/temp/'+str(imgname)))
    cv2_imshow(img)
    # img = cv2.resize(img,(320,240))
    # # cv2_imshow(img)
    # img = cv2.resize(img,(160,120))
    # cv2_imshow(img)
    # img = cv2.resize(img,(80,60))
    # cv2_imshow(img)
    # cv2_imshow(img)
    # print(img.shape)
    orig_size = img.shape[:2][::-1]
    inter_img = (img*img_scale - img_mean)/img_std
    img_inp = torch.tensor(inter_img.transpose(2, 0, 1)[None]).float()
    img_inp = img_inp.cuda()
    seg = net(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
    seg = cv2.resize(seg, orig_size, interpolation=cv2.INTER_CUBIC)
    seg = cmap[seg.argmax(axis=2).astype(uint8)]
    ma = (seg[:,:,0]>=0) & (seg[:,:,0]<=20) & (seg[:,:,2]>=110) & (seg[:,:,2]<=150) & (seg[:,:,1]>=0) & (seg[:,:,1]<=20)
    x = zeros(ma.shape)
    x[ma==True] = 1
    x[ma==False] = 0
    set_printoptions(threshold=inf)
    # print(x[479])
    cv2.imwrite('/content/480_640/'+str(imgname),x)
    # w.write_to_csv(x, '/content/drive/MyDrive/handmask/sample_240_320.csv')

In [ ]:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]

In [ ]:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]

In [ ]:
!zip -r /content/60_80.zip /content/60_80

In [ ]:
!zip -r /content/120_160.zip /content/120_160

In [ ]:
!zip -r /content/240_320.zip /content/240_320

In [ ]:
!unzip /content/drive/MyDrive/handmask/Age,Bird,Favourite_handmask.zip